# MQTT Client Subscriber
---

Mqtt subscriber for exercise 1.b of homework 3

In [1]:
!pip install redis
!pip install paho-mqtt

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


Create Redis Manager

In [2]:
REDIS_HOST = 'redis-11747.c300.eu-central-1-1.ec2.cloud.redislabs.com'
REDIS_PORT = 11747
REDIS_PASSWORD = 'i4Wds8wGEcQ5PVQwBMmEQL17GH4XdXKH'


In [3]:
import redis
from math import floor


class RedisManager:
    """ REDIS Manager Class for homework3 ML4IoT """
    def __init__(self, _hostname: str = REDIS_HOST, _port: int = REDIS_PORT, _password: str = REDIS_PASSWORD):
        self.client = None
        try:
            self.client = redis.Redis(host=_hostname, port=_port, password=_password)
            is_connected = self.client.ping()
            print('Redis client is connected ', is_connected)
        except Exception as ex:
            print(ex)
        if self.client is None:
            exit(1)
        self.address = ''

    def init_timeseries(self, address):
        self.address = address
        self.client.delete(f'{address}:battery')
        self.client.delete(f'{address}:power')
        self.client.delete(f'{address}:plugged')


        self.client.ts().create(f'{address}:battery')
        self.client.ts().create(f'{address}:power')
        self.client.ts().create(f'{address}:plugged')


    def save_data(self, battery, power, tmp):
        self.client.ts().add(f'{self.address}:battery', int(tmp), battery)
        self.client.ts().add(f'{self.address}:power', int(tmp), power)
        return


In [4]:
import uuid

mac_address = hex(uuid.getnode())

redis_client = RedisManager()
redis_client.init_timeseries(mac_address)

Redis client is connected  True


Create MqttClient 

In [5]:
TOPIC = 's290453'

# Used test.mosquitto.org instead of mqtt.eclipse.io, because it gives some connectivity problems
BROKER = 'test.mosquitto.org'
PORT = 1883
KEEP_ALIVE = 60

In [6]:
import paho.mqtt.client as mqtt
import json

def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))
    client.subscribe(TOPIC)


def on_message(client, userdata, msg):
    print(msg.topic + " "+ str(msg.payload) )
    data = json.loads(msg.payload)
    redis_client.save_data(battery=data["battery_level"], power=data["power_plugged"], tmp=data["timestamp"])


client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message
client.connect(BROKER, PORT, KEEP_ALIVE)

client.loop_forever()


Connected with result code 0
s290453 b'{"mac_address": "0xc025a546d17b", "timestamp": 1673890031.1109369, "battery_level": 55, "power_plugged": 1}'
s290453 b'{"mac_address": "0xc025a546d17b", "timestamp": 1673890032.1168377, "battery_level": 55, "power_plugged": 1}'
s290453 b'{"mac_address": "0xc025a546d17b", "timestamp": 1673890033.1307626, "battery_level": 55, "power_plugged": 1}'
s290453 b'{"mac_address": "0xc025a546d17b", "timestamp": 1673890034.1419988, "battery_level": 55, "power_plugged": 1}'
s290453 b'{"mac_address": "0xc025a546d17b", "timestamp": 1673890035.1491396, "battery_level": 55, "power_plugged": 1}'
s290453 b'{"mac_address": "0xc025a546d17b", "timestamp": 1673890036.158418, "battery_level": 55, "power_plugged": 1}'
s290453 b'{"mac_address": "0xc025a546d17b", "timestamp": 1673890037.1586685, "battery_level": 55, "power_plugged": 0}'
s290453 b'{"mac_address": "0xc025a546d17b", "timestamp": 1673890038.1634018, "battery_level": 55, "power_plugged": 0}'
s290453 b'{"mac_addr

KeyboardInterrupt: 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=82f6f9dd-e2eb-4faf-a867-1a0a5dea3d6b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>